In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv("SCOA_A4.csv")

# Identify feature and target columns
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y_categorical = df['species']

# Encode the target column
le = LabelEncoder()
y = le.fit_transform(y_categorical)

print("Dataset loaded successfully.")
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print("First 5 rows of features (X):\n", X.head())
print("First 5 encoded target values (y):\n", y[:5])
print("Original species labels (first 5):\n", y_categorical[:5])
print("Encoded target classes:", le.classes_)

import numpy as np
import random
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# --- Genetic Algorithm Setup ---
POP_SIZE = 20      # number of individuals
N_GENERATIONS = 10 # iterations
MUTATION_RATE = 0.2

# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]

def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   random_state=42) # Added random_state for reproducibility
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()

def selection(population, fitnesses):
    # Select the best two individuals for crossover
    idx = np.argsort(fitnesses)[-2:]
    return [population[idx[0]], population[idx[1]]]

def crossover(parent1, parent2):
    # Perform single-point crossover
    point = random.randint(0, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):
    # Apply mutation to max_depth
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    # Apply mutation to min_samples_split
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome

# --- Run GA ---
population = [create_chromosome() for _ in range(POP_SIZE)]

print("\nStarting Genetic Algorithm...")
for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen+1}/{N_GENERATIONS} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)

    # Generate new population through crossover and mutation
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population

# Best result from the final population
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
best_hyperparameters = population[best_idx]
best_accuracy = fitnesses[best_idx]

print("\nGenetic Algorithm Finished.")
print(f"Best Hyperparameters: max_depth={best_hyperparameters[0]}, min_samples_split={best_hyperparameters[1]}")
print(f"Best Accuracy: {best_accuracy:.4f}")

Dataset loaded successfully.
Features (X) shape: (150, 4)
Target (y) shape: (150,)
First 5 rows of features (X):
    sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
First 5 encoded target values (y):
 [0 0 0 0 0]
Original species labels (first 5):
 0    setosa
1    setosa
2    setosa
3    setosa
4    setosa
Name: species, dtype: object
Encoded target classes: ['setosa' 'versicolor' 'virginica']

Starting Genetic Algorithm...
Generation 1/10 - Best Fitness: 0.9733
Generation 2/10 - Best Fitness: 0.9733
Generation 3/10 - Best Fitness: 0.9733
Generation 4/10 - Best Fitness: 0.9733
Generation 5/10 - Best Fitness: 0.9733
Generation 6/10 - Best Fitness: 0.9733
Generation 7/10 - Best Fitness: 0.9733
Generation 8/

In [ ]:
# ------------------------------------------------------------------------
# Step-by-step code explanation (simple language) — paste above your code
# ------------------------------------------------------------------------
# 1) Import libraries
#    - pandas: for reading and handling the dataset (tables, rows, columns).
#    - sklearn.preprocessing.LabelEncoder: to convert text labels (species names)
#      into numeric labels that machine learning models can use.
#
# 2) Load the dataset
#    - df = pd.read_csv("SCOA_A4.csv")
#    - This reads the CSV file into a pandas DataFrame named df.
#
# 3) Identify features and target
#    - X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
#      These four columns are used as the input features for the model.
#    - y_categorical = df['species']
#      This column (species names) is the target variable we want to predict.
#
# 4) Encode the target (LabelEncoder)
#    - le = LabelEncoder()
#    - y = le.fit_transform(y_categorical)
#    - This converts species names (strings) into integers (e.g., 'setosa' -> 0).
#      We keep the mapping in le.classes_; printing it shows which integer
#      corresponds to which species name.
#
# 5) Print basic dataset info
#    - The script prints confirmation and shapes:
#      - "Dataset loaded successfully."
#      - "Features (X) shape: (n_samples, 4)"
#      - "Target (y) shape: (n_samples,)"
#      - Prints first rows so you can sanity-check the data.
#
# 6) Import additional libraries for model and evaluation
#    - numpy, random: used for GA operations and array handling.
#    - cross_val_score: to evaluate model performance using k-fold cross-validation.
#    - DecisionTreeClassifier: the model used; its hyperparameters are tuned by the GA.
#
# 7) Genetic Algorithm (GA) setup
#    - POP_SIZE = 20: number of candidate solutions (individuals) in each generation.
#    - N_GENERATIONS = 10: number of GA iterations (how many times population evolves).
#    - MUTATION_RATE = 0.2: probability of random changes in offspring.
#    - Chromosome (representation): [max_depth, min_samples_split]
#      - max_depth: how deep the decision tree may grow.
#      - min_samples_split: minimum number of samples required to split an internal node.
#
# 8) create_chromosome()
#    - Builds a random candidate hyperparameter pair:
#      - max_depth randomly chosen from 1 to 20.
#      - min_samples_split randomly chosen from 2 to 10.
#
# 9) fitness(chromosome)
#    - For a given chromosome (hyperparams), this function:
#      - Builds a DecisionTreeClassifier with those hyperparameters.
#      - Runs cross_val_score(model, X, y, cv=5) to estimate the model's average accuracy.
#      - Returns the mean cross-validation accuracy as the fitness value
#        (higher is better).
#
# 10) selection(population, fitnesses)
#    - Chooses two best individuals from the population (highest fitness) to act as parents.
#
# 11) crossover(parent1, parent2)
#    - Single-point crossover:
#      - Choose a cut point and swap parts to create two children.
#      - Ensures children inherit hyperparameters from both parents.
#
# 12) mutate(chromosome)
#    - Randomly modifies a chromosome with small probability (mutation).
#      - Example: change max_depth or min_samples_split randomly.
#    - Mutation helps maintain diversity and lets GA explore new regions.
#
# 13) Main GA loop
#    - Initialize a random population of POP_SIZE chromosomes.
#    - For each generation (repeat N_GENERATIONS times):
#      - Evaluate fitness for each individual.
#      - Select parents (best two).
#      - Create offspring via crossover and mutation until new_population is filled.
#      - Replace population with new_population.
#
# 14) Final result
#    - After the GA finishes, compute fitnesses for final population.
#    - Select the individual with the highest fitness.
#    - Print the best hyperparameters and the best accuracy observed:
#      - Example printed lines:
#        "Genetic Algorithm Finished."
#        "Best Hyperparameters: max_depth=..., min_samples_split=..."
#        "Best Accuracy: 0.XXXX"
#
# ------------------------------------------------------------------------
# How to interpret the printed outputs (what to say in viva)
# ------------------------------------------------------------------------
# - "Dataset loaded successfully." simply means the CSV was found and read.
# - Features (X) shape: (n, 4) means there are n examples and 4 input features.
# - Target (y) shape: (n,) means there are n label values.
# - The printed "first 5 rows" and encoded labels let you check:
#     - Values are reasonable (no missing garbage),
#     - Label encoding mapping (le.classes_) shows original class names in order.
# - During GA execution you typically won't see per-generation output unless printed,
#   but the final "Best Accuracy" is the mean 5-fold CV accuracy for the best hyperparameters.
# - Example interpretation: "Best Accuracy: 0.92" → the Decision Tree with the chosen
#   hyperparameters achieved on average 92% accuracy across the 5 folds.
#   It means the model predicted the correct species 92% of the time in cross-validation.

In [ ]:
# -------------------------------------------------------------------------
# Detailed theory of Genetic Algorithms (GA) — comprehensive, viva-ready
# Paste this as comments into a Jupyter cell or read aloud during viva.
# -------------------------------------------------------------------------
#
# 1) What is a Genetic Algorithm (high-level)
# -------------------------------------------
# - A Genetic Algorithm (GA) is a population-based metaheuristic optimization
#   method inspired by natural selection and genetics (biological evolution).
# - It searches for good solutions to optimization/search problems by evolving
#   a population of candidate solutions through repeated application of:
#     Selection -> Crossover (recombination) -> Mutation.
# - GAs are useful for problems with large, complex, discontinuous, or
#   non-differentiable search spaces where gradient-based methods fail.
#
# 2) Historical note (one-liner)
# ------------------------------
# - Developed from ideas in the 1960s–1970s (notably John Holland); widely used
#   since then across engineering, computer science, and applied sciences.
#
# 3) Core components of a GA
# --------------------------
# - Representation / Encoding:
#   - How candidate solutions (individuals) are encoded as chromosomes (strings).
#   - Common encodings:
#       * Binary encoding: chromosome is bitstring (e.g., '101001').
#       * Real-valued (floating point) encoding: genes are floats (good for continuous).
#       * Permutation encoding: used for ordering problems (TSP); chromosome is a permutation.
#       * Tree encoding: used in genetic programming (programs as trees).
#
# - Population:
#   - A set of candidate solutions. Size is a hyperparameter (POP_SIZE).
#   - Larger populations increase diversity but cost more evaluations.
#
# - Fitness Function:
#   - Maps a candidate solution to a scalar that measures solution quality.
#   - GA seeks to maximize (or minimize) fitness. For minimization, use negated loss.
#   - Fitness must reflect true objective and constraints (often via penalties).
#
# - Selection Operators:
#   - Choose parents for reproduction based on fitness (fitter individuals more likely).
#   - Common methods:
#       * Roulette-wheel selection (fitness-proportionate): probability ∝ fitness.
#       * Ranking selection: individuals ranked; selection probability based on rank.
#       * Tournament selection: pick k random individuals and select the best among them.
#       * Stochastic universal sampling: similar to roulette but lower variance.
#   - Tradeoffs: roulette can be dominated by very fit individuals; tournament is robust.
#
# - Crossover (Recombination):
#   - Combine genetic material of two parents to create offspring.
#   - Types:
#       * Single-point crossover: split at one point, swap tails.
#       * Two/multi-point crossover: split at multiple points.
#       * Uniform crossover: each gene is chosen from either parent with a probability.
#       * Arithmetic crossover (real-valued): offspring = α * parent1 + (1-α) * parent2.
#       * Order crossover (permutation problems): preserves ordering constraints (e.g., OX for TSP).
#   - Purpose: mix useful building blocks from parents to create potentially better offspring.
#
# - Mutation:
#   - Randomly change genes to maintain diversity and explore new regions.
#   - Types:
#       * Bit-flip mutation (binary): flip bits with small probability.
#       * Gaussian perturbation (real-valued): add N(0,σ) noise to genes.
#       * Swap/inversion mutation (permutation): swap two positions or invert a subsequence.
#   - Mutation rate is typically low (e.g., 0.001–0.1 depending on encoding).
#
# - Replacement / Survivor Selection:
#   - Decide who survives to the next generation.
#   - Strategies:
#       * Generational replacement: all parents replaced by children.
#       * Steady-state: only some individuals replaced each generation.
#       * Elitism: best individual(s) always carried forward to avoid losing best found solution.
#
# 4) Pseudocode (standard GA)
# ---------------------------
# - population = initialize_random_population(POP_SIZE)
# - evaluate fitness for all individuals
# - while termination_condition_not_met:
#     parents = select_parents(population, fitness)
#     offspring = []
#     while len(offspring) < POP_SIZE:
#         p1, p2 = pick_two_parents(parents)
#         if random() < CROSSOVER_RATE:
#             c1, c2 = crossover(p1, p2)
#         else:
#             c1, c2 = p1.copy(), p2.copy()
#         mutate(c1); mutate(c2)
#         offspring.extend([c1, c2])
#     population = replace(population, offspring, fitness)    # may include elitism
#     evaluate fitness for any new individuals
# - return best individual found

#
# 9) Parameter settings & practical guidelines
# -------------------------------------------
# - Population size (POP_SIZE):
#   - Typical range: 20–500. Larger for complex or multi-modal problems.
#   - Tradeoff: larger populations -> better coverage but more evaluations.
# - Crossover rate (CR):
#   - Often high: 0.6–1.0 for classical GAs (commonly 0.8–0.9).
# - Mutation rate (MR):
#   - Usually low for binary (0.001–0.01 per bit), higher for real-valued genes (0.01–0.1).
# - Number of generations:
#   - Depends on problem complexity and evaluation budget; use convergence criteria:
#     * Max generations reached OR fitness improvement below threshold for N gens.
# - Selection pressure:
#   - Controls speed of convergence: high pressure -> fast convergence, risk of premature convergence.
#   - Tune tournament size or ranking parameters to adjust pressure.
# - Use elitism (1-5 individuals) to avoid losing best solutions.
# - Always run GA multiple times with different random seeds — it is stochastic.
#
# 10) Convergence & stopping criteria
# -----------------------------------
# - Common choices:
#   - Fixed number of generations or fitness evaluations.
#   - No improvement in best fitness for T successive generations.
#   - Achieve target fitness threshold.
# - Note: reaching identical individuals in population often means convergence but not necessarily global optimum.
#
# 11) Computational complexity & evaluation cost
# ----------------------------------------------
# - The dominant cost is fitness evaluation (often involves simulations or ML training).
# - Parallelization:
#   - Fitness evaluations for individuals are independent — easy to parallelize.
#   - Use parallel or distributed computing to reduce wall-clock time.
#
# 12) Strengths of GAs
# --------------------
# - Global search capability; less likely to be trapped by local minima compared to simple hill-climbers.
# - Flexible: works with discrete, continuous, combinatorial, or mixed-variable problems.
# - Does not require gradient information or convexity.
# - Naturally parallelizable.
#
# 13) Weaknesses / pitfalls
# -------------------------
# - Computationally expensive if fitness is costly (many evaluations required).
# - Stochastic: results vary between runs; needs multiple runs for reliability.
# - Requires careful tuning of population size, mutation/crossover rates, and selection pressure.
# - Poorly designed encoding or operators can make search ineffective (e.g., crossover destroys structure).
# - Schema theory gives intuition but not strict guarantees of finding global optimum.
#
# 14) Common variants & hybrid approaches
# ---------------------------------------
# - Genetic programming (GP): evolves programs or expressions (tree-structured chromosomes).
# - Evolution strategies (ES) and Differential Evolution (DE): related real-valued EAs with special mutation/recombination.
# - Estimation of Distribution Algorithms (EDA): build probabilistic models of promising solutions.
# - Memetic algorithms: hybridize GA with local search (hill-climbing) to refine offspring.
# - Island models: multiple subpopulations occasionally exchange individuals (migration) to improve diversity.
#
# 15) Implementation tips (practical)
# ----------------------------------
# - Normalize/scale real-valued genes if needed; choose appropriate mutation distributions.
# - Implement elitism to protect the best-so-far solution.
# - Log best/mean/worst fitness per generation for diagnosing progress.
# - Visualize convergence (plot best fitness vs generation).
# - Use reproducible randomness (set random seeds) for debugging/demonstrations, but remember true performance needs multiple seeds.
# - If fitness evaluation is expensive, consider surrogate models (approximate fitness with a cheaper model) or asynchronous parallel evaluation.
# - Use tournament selection for simplicity and robustness.
# - For permutation problems (TSP, scheduling), avoid standard crossover/mutation that break feasibility — use order-preserving operators.
#
# 16) Example use-cases (short)
# -----------------------------
# - Hyperparameter tuning (ML models), scheduling, routing (TSP), design optimization (engineering),
#   feature selection, symbolic regression (GP), evolving neural network architectures (neuroevolution).
#
# 17) How to present GA in a viva (concise bullets)
# ------------------------------------------------
# - "GA maintains a population of candidate solutions and uses selection, crossover, and mutation
#    to explore and exploit the search space."
# - "Fitness guides selection; crossover recombines good partial solutions; mutation introduces novelty."
# - "Key hyperparameters: population size, crossover rate, mutation rate, number of generations."
# - "GAs are stochastic and require multiple runs; they are powerful for complex, non-differentiable problems."
#
# 18) Short mathematical notations (if asked)
# -------------------------------------------
# - Let P(t) be population at generation t.
# - Fitness f : solution -> ℝ maps solutions to objective values.
# - Expected number of copies of individual i in next gen under fitness-proportionate selection:
#     E[copies_i] = (f_i / f̄) ,   where f̄ is mean fitness of population.
# - Schema theorem (informal expected propagation):
#     E[m(H,t+1)] ≥ m(H,t) * (f(H)/f̄) * (1 - p_c * (δ(H)/(l-1)) - o(H) * p_m)
#   where m(H,t) = number of individuals matching schema H at generation t,
#   f(H) = average fitness of schema H, δ(H) = defining length, o(H)=order of H,
#   p_c = crossover probability, p_m = mutation probability. (This is conceptual; you can skip algebra.)
#
